In [ ]:
# Code adapted from UIUC PS 590: Images and Text as Data
# by Prof. Nora Webb Williams. She adapted from adapted 
# from Diyi Yang and crew at Georgia Tech.
import ktrain
import pandas as pd

import ktrain
from ktrain import text as ktext
from sklearn.model_selection import train_test_split
from sklearn import metrics

df = pd.read_csv("data.csv")
text = df['text'].to_list()
# TODO: Text pre-processing?
labels = df['label'].to_list()
target_name = ['Not islamophobic', 'Islamophobic']
# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(text, labels, random_state = 0)
# Preprocess data
trn, val, preproc = ktext.texts_from_array(x_train=X_train, y_train=y_train,
                                          x_test=X_test, y_test=y_test,
                                          class_names = target_name,
                                          preprocess_mode='distilbert',
                                          maxlen=128)
# Building a model and a learner
model = ktext.text_classifier('distilbert', train_data=trn, preproc=preproc)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=16)
# Train the model
learner.fit_onecycle(3e-5, 4)

# Predict on test set
p = ktrain.get_predictor(model, preproc)
y_pred = p.predict(X_test)

y_pred_num = []
for pred in y_pred:
    rel_index = target_name.index(pred)
    y_pred_num.append(rel_index)
# Print model metrics
print(metrics.classification_report(y_test, y_pred_num))